<a href="https://colab.research.google.com/github/nanase1025/M-SAM/blob/main/nb/Qwen2_5_7B_VL_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

### Unsloth

We're also introducing how you can do `GSPO` inside of Unsloth as well!

The goal of this notebook is to make a vision language model solve maths problems via reinforcement learning given an image input like below:

<img src="https://raw.githubusercontent.com/lupantech/MathVista/main/assets/our_new_3_datasets.png" alt="Alt text" height="256">

In [3]:
from unsloth import FastVisionModel
import torch
max_seq_length = 16384 # Must be this long for VLMs
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastVisionModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-VL-7B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-07 01:43:56 [__init__.py:244] Automatically detected platform cuda.
ERROR 10-07 01:43:59 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.55.4. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
INFO 10-07 01:44:13 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
INFO 10-07 01:44:13 [vllm_utils.py:717] Unsloth: Patching vLLM v0 graph capture
Unsloth: 

preprocessor_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

INFO 10-07 01:44:30 [config.py:841] This model supports multiple tasks: {'classify', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 10-07 01:44:30 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 10-07 01:44:30 [config.py:1472] Using max model len 16384
WARNING 10-07 01:44:30 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 10-07 01:44:30 [arg_utils.py:1556] --enable-prefix-caching is not supported for multimodal models in V0 and has been disabled.
INFO 10-07 01:44:32 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['embed_tokens', 'em

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

video_preprocessor_config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

INFO 10-07 01:47:49 [worker.py:294] Memory profiling takes 59.36 seconds
INFO 10-07 01:47:49 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.77) = 11.34GiB
INFO 10-07 01:47:49 [worker.py:294] model weights take 6.79GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 2.71GiB; the rest of the memory reserved for KV Cache is 1.81GiB.
INFO 10-07 01:47:50 [executor_base.py:113] # cuda blocks: 2118, # CPU blocks: 0
INFO 10-07 01:47:50 [executor_base.py:118] Maximum concurrency for 16384 tokens per request: 2.07x
INFO 10-07 01:47:50 [vllm_utils.py:722] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 10-07 01:47:50 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `

Capturing CUDA graph shapes:   0%|          | 0/1 [00:00<?, ?it/s]

INFO 10-07 01:47:57 [model_runner.py:1671] Graph capturing finished in 7 secs, took 0.04 GiB
INFO 10-07 01:47:57 [vllm_utils.py:729] Unsloth: Patched vLLM v0 graph capture finished in 7 secs.
INFO 10-07 01:47:58 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 68.79 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'post_layernorm', 'norm2', 'input_layernorm', 'ffn_norm', 'layer_norm2', 'attention_norm', 'post_attention_layernorm', 'q_norm', 'pre_feedforward_layernorm', 'layer_norm1', 'k_norm', 'norm1']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'post_layernorm', 'input_layernorm', 'ffn_norm', 'layer_norm2', 'attention_norm', 'post_attention_layernorm', 'cross_attn_input_layernorm', 'q_norm', 'pre_feedforward_layernorm', 'layer_norm1', 'k_norm', 'cross_attn_post_attention_layernorm']


In Unsloth, we share vLLM's weights directly, reducing VRAM usage by > 50%. vLLM also does not yet support LoRA on the vision layers, so we can only add them on the language layers. Vision GRPO still works though!

In [4]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True,  # False if not finetuning language layers
    finetune_attention_modules = True,  # False if not finetuning attention layers
    finetune_mlp_modules       = True,  # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [5]:
! pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
absolufy-imports                         0.3.1
accelerate                               1.10.1
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.0
aiosignal                                1.4.0
airportsdata                             20250909
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.16.5
altair                                   5.5.0
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.11.0
anywidget                                0.9.18
argon2-cffi                      

### Data Prep
<a name="Data"></a>

`AI4Math/MathVista` is a dataset that involves using images to solve logic and math problems.

For this notebook, we will only use math problems with numeric answers for simpilicity.

In [6]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

dataset = load_dataset("AI4Math/MathVista", split = "testmini")

README.md: 0.00B [00:00, ?B/s]

data/testmini-00000-of-00001-725687bf7a1(…):   0%|          | 0.00/142M [00:00<?, ?B/s]

data/test-00000-of-00002-6b81bd7f7e2065e(…):   0%|          | 0.00/358M [00:00<?, ?B/s]

data/test-00001-of-00002-6a611c71596db30(…):   0%|          | 0.00/386M [00:00<?, ?B/s]

Generating testmini split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5141 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'pid': '1',
 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?",
 'image': 'images/1.jpg',
 'decoded_image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1514x720>,
 'choices': None,
 'unit': None,
 'precision': 1.0,
 'answer':

We filter the dataset to keep only float or numeric answers:

In [8]:
def is_numeric_answer(example):
    try:
        float(example["answer"])
        return True
    except:
        return False

dataset = dataset.filter(is_numeric_answer)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

We also resize the images to be 512 by 512 pixels to make the images managable in context length. We also convert them to RGB so they are compatible for training!

In [9]:
# Resize to (512, 512)
def resize_images(example):
    image = example["decoded_image"]
    image = image.resize((512, 512))
    example["decoded_image"] = image
    return example
dataset = dataset.map(resize_images)

# Then convert to RGB
def convert_to_rgb(example):
    image = example["decoded_image"]
    if image.mode != "RGB":
        image = image.convert("RGB")
    example["decoded_image"] = image
    return example
dataset = dataset.map(convert_to_rgb)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

We then create the conversational template that is needed to collate the dataset for RL:

In [10]:
# Define the delimiter variables for clarity and easy modification
REASONING_START = "<REASONING>"
REASONING_END = "</REASONING>"
SOLUTION_START = "<SOLUTION>"
SOLUTION_END = "</SOLUTION>"

def make_conversation(example):
    # Define placeholder constants if they are not defined globally
    # The user's text prompt
    text_content = (
        f"{example['question']}. Also first provide your reasoning or working out"\
        f" on how you would go about solving the question between {REASONING_START} and {REASONING_END}"
        f" and then your final answer between {SOLUTION_START} and (put a single float here) {SOLUTION_END}"
    )

    # Construct the prompt in the desired multi-modal format
    prompt = [
        {
            "role": "user",
            "content": [
                {"type": "image"},  # Placeholder for the image
                {"type": "text", "text": text_content},  # The text part of the prompt
            ],
        },
    ]
    # The actual image data is kept separate for the processor
    return {"prompt": prompt, "image": example["decoded_image"], "answer": example["answer"]}

train_dataset = dataset.map(make_conversation)

# We're reformatting dataset like this because decoded_images are the actual images
# The "image": example["decoded_image"] does not properly format the dataset correctly

# 1. Remove the original 'image' column
train_dataset = train_dataset.remove_columns("image")

# 2. Rename 'decoded_image' to 'image'
train_dataset = train_dataset.rename_column("decoded_image", "image")

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

Now let's apply the chat template across the entire dataset:

In [11]:
train_dataset = train_dataset.map(
    lambda example: {
        "prompt": tokenizer.apply_chat_template(
            example["prompt"],
            tokenize = False,
            add_generation_prompt = True, # Must add assistant
        )
    }
)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

## Reward functions

We now define some basic formatting rewards functions to see if reasoning starts and ends, and also another to see if the answers were written correctly.

We also try to fix the `addCriterion` issue as described in our [blog post](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl#qwen-2.5-vl-vision-rl-issues-and-quirks)

In [12]:
# Reward functions
import re

def formatting_reward_func(completions,**kwargs):
    import re
    thinking_pattern = f'{REASONING_START}(.*?){REASONING_END}'
    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    scores = []
    for completion in completions:
        score = 0
        thinking_matches = re.findall(thinking_pattern, completion, re.DOTALL)
        answer_matches = re.findall(answer_pattern, completion, re.DOTALL)
        if len(thinking_matches) == 1:
            score += 1.0
        if len(answer_matches) == 1:
            score += 1.0

        # Fix up addCriterion issues
        # See https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl#qwen-2.5-vl-vision-rl-issues-and-quirks
        # Penalize on excessive addCriterion and newlines
        if len(completion) != 0:
            removal = completion.replace("addCriterion", "").replace("\n", "")
            if (len(completion)-len(removal))/len(completion) >= 0.5:
                score -= 2.0

        scores.append(score)
    return scores


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    responses = [re.findall(answer_pattern, completion, re.DOTALL) for completion in completions]
    q = prompts[0]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:{completions[0]}")
    return [
        2.0 if len(r)==1 and a == r[0].replace('\n','') else 0.0
        for r, a in zip(responses, answer)
    ]

Here is the first example prompt in the dataset

In [13]:
train_dataset[0]["prompt"]

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?. Also first provide your reasoning or working out on how you would go about so

<a name="Inference"></a>
### Inference
Now let's try the model on the hundredth sample of the train dataset without training.


In [14]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)

outputs = model.fast_generate(
    {
        "prompt": train_dataset[100]["prompt"],
        "multi_modal_data": {"image": train_dataset[100]["image"]}
    },
    sampling_params,
)
print(outputs[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<REASONING>
To measure the length of the nail, we need to align it with the ruler and observe where it ends relative to the markings on the ruler.

1. Place the nail on the ruler so that the tip of the nail is at the 0-inch mark.
2. Observe where the back end of the nail falls on the ruler.
3. The back end of the nail appears to be just past the 3-inch mark but not quite reaching the 4-inch mark.

Since the question asks for the length to the nearest inch, we need to determine if the nail is closer to 3 inches or 4 inches in length. In this case, the nail is closer to 3 inches than to 4 inches because the back end of the nail is closer to the 3-inch mark than the 4-inch mark.

Therefore, the nail is about 3 inches long.
</REASONING>

<SOLUTION>
3</SOLUTION>


In [16]:
train_dataset[0].keys()
train_dataset[0]['answer']

'1.2'

<a name="Train"></a>
### Train the model

Now set up the `GRPO` Trainer and all configurations! Note we actually enable `GSPO` as well!

In [17]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    log_completions = False,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    num_train_epochs = 0.5, # Set to 1 for a full training run
    # max_steps = 60,
    save_steps = 60,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # Below enables GSPO:
    importance_sampling_level = "sequence",
    mask_truncated_completions = False,
    loss_type = "dr_grpo",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |

During inference, you might encounter `addCriterion` or some weird gibberish outputs. Please read our [blog post](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl#qwen-2.5-vl-vision-rl-issues-and-quirks) on why this occurs. It seems to be an inherent thing inside of the model, and we can ignore this.

In [ ]:
trainer = GRPOTrainer(
    model = model,
    args = training_args,
    # Pass the processor to handle multimodal inputs
    processing_class = tokenizer,
    reward_funcs = [
        formatting_reward_func,
        correctness_reward_func,
    ],
    train_dataset = train_dataset,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 566 | Num Epochs = 1 | Total steps = 142
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 8,332,536,832 (0.48% trained)


-------------------- Question:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>What is the highest value on the X axis?. Also first provide your reasoning or working out on how you would go about solving the question between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a single float here) </SOLUTION><|im_end|>
<|im_start|>assistant
 
Answer:
30 
Response:<REASONING>
To determine the highest value on the X-axis, I need to look at the numerical markings on the horizontal axis, which represents the "MICROGRAMS/mL-E-DNP-LYSIME-HCL" on the figure.

The X-axis is labeled with numerical values from 0 to 30, with increments of 5. The highest value marked on the X-axis is 30.

</REASONING>

<SOLUTION>
30.0 </SOLUTION>
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / formatting_reward_func / mean,rewards / formatting_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.030200,1.750000,0.288675,182.750000,100.000000,302.000000,0.000000,182.750000,100.000000,302.000000,0.000010,1.750000,0.462910,0.000000,0.000000
2,-0.010900,1.750000,0.500000,188.625000,131.000000,268.000000,0.000000,188.625000,131.000000,268.000000,0.000000,1.750000,0.707107,0.000000,0.000000
3,0.048700,1.875000,0.250000,203.750000,133.000000,309.000000,0.000000,203.750000,133.000000,309.000000,0.000007,1.875000,0.353553,0.000000,0.000000
4,0.061000,2.000000,0.788675,257.500000,123.000000,508.000000,0.000000,257.500000,123.000000,508.000000,0.000008,1.750000,0.462910,0.250000,0.707107
5,-0.021100,1.875000,0.250000,218.500000,151.000000,320.000000,0.000000,218.500000,151.000000,320.000000,0.000009,1.875000,0.353553,0.000000,0.000000
6,0.089100,1.375000,0.767389,259.000000,103.000000,360.000000,0.000000,259.000000,103.000000,360.000000,0.000015,1.375000,0.744024,0.000000,0.000000
7,0.013700,1.750000,0.500000,206.000000,127.000000,262.000000,0.000000,206.000000,127.000000,262.000000,0.000008,1.750000,0.462910,0.000000,0.000000
8,0.000000,2.000000,0.000000,124.000000,115.000000,136.000000,0.000000,124.000000,115.000000,136.000000,0.000006,2.000000,0.000000,0.000000,0.000000
9,0.068700,1.875000,0.250000,259.625000,137.000000,441.000000,0.000000,259.625000,137.000000,441.000000,0.000008,1.875000,0.353553,0.000000,0.000000
10,0.036800,1.625000,0.478714,254.750000,103.000000,416.000000,0.000000,254.750000,103.000000,416.000000,0.000007,1.625000,0.744024,0.000000,0.000000


-------------------- Question:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Subtract all yellow metallic balls. Subtract all small yellow shiny things. How many objects are left?. Also first provide your reasoning or working out on how you would go about solving the question between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a single float here) </SOLUTION><|im_end|>
<|im_start|>assistant
 
Answer:
8 
Response:<REASONING>
1. Identify all yellow metallic balls in the image. There is one yellow metallic ball.
2. Identify all small yellow shiny things. There are two small yellow shiny objects: a yellow metallic ball and a yellow metallic cylinder.
3. Subtract these objects from the total count of objects in the image.
   - Total objects in the image: 10
   - Subtract yellow metallic ball: 9
   - Subtract yellow metallic cylinder: 8

Therefore, the number of objects left after 

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / formatting_reward_func / mean,rewards / formatting_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.030200,1.750000,0.288675,182.750000,100.000000,302.000000,0.000000,182.750000,100.000000,302.000000,0.000010,1.750000,0.462910,0.000000,0.000000
2,-0.010900,1.750000,0.500000,188.625000,131.000000,268.000000,0.000000,188.625000,131.000000,268.000000,0.000000,1.750000,0.707107,0.000000,0.000000
3,0.048700,1.875000,0.250000,203.750000,133.000000,309.000000,0.000000,203.750000,133.000000,309.000000,0.000007,1.875000,0.353553,0.000000,0.000000
4,0.061000,2.000000,0.788675,257.500000,123.000000,508.000000,0.000000,257.500000,123.000000,508.000000,0.000008,1.750000,0.462910,0.250000,0.707107
5,-0.021100,1.875000,0.250000,218.500000,151.000000,320.000000,0.000000,218.500000,151.000000,320.000000,0.000009,1.875000,0.353553,0.000000,0.000000
6,0.089100,1.375000,0.767389,259.000000,103.000000,360.000000,0.000000,259.000000,103.000000,360.000000,0.000015,1.375000,0.744024,0.000000,0.000000
7,0.013700,1.750000,0.500000,206.000000,127.000000,262.000000,0.000000,206.000000,127.000000,262.000000,0.000008,1.750000,0.462910,0.000000,0.000000
8,0.000000,2.000000,0.000000,124.000000,115.000000,136.000000,0.000000,124.000000,115.000000,136.000000,0.000006,2.000000,0.000000,0.000000,0.000000
9,0.068700,1.875000,0.250000,259.625000,137.000000,441.000000,0.000000,259.625000,137.000000,441.000000,0.000008,1.875000,0.353553,0.000000,0.000000
10,0.036800,1.625000,0.478714,254.750000,103.000000,416.000000,0.000000,254.750000,103.000000,416.000000,0.000007,1.625000,0.744024,0.000000,0.000000


<a name="Inference"></a>
### Inference



And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_lora")

We try calling vLLM with our trained RL model:

In [ ]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)

outputs = model.fast_generate(
    {
        "prompt": train_dataset[165]["prompt"],
        "multi_modal_data": {"image": train_dataset[165]["image"]}
    },
    sampling_params,
    lora_request = model.load_lora("grpo_lora"))
print(outputs[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

To solve for the magnitude of the average force on the driver during the collision, we need to use the impulse-momentum theorem. The impulse-momentum theorem states that the impulse (force over time) is equal to the change in momentum.

Given:
- Initial velocity \( v_i = 70 \, \text{m/s} \) at \( 30^\circ \) from the wall.
- Final velocity \( v_f = 50 \, \text{m/s} \) at \( 10^\circ \) from the wall.
- Mass \( m = 80 \, \text{kg} \).
- Duration of the collision \( \Delta t = 14 \, \text{ms} = 14 \times 10^{-3} \, \text{s} \).

### Reasoning:

1. **Calculate the initial momentum \( \mathbf{p}_i \):**
   \[
   \mathbf{p}_i = m v_i \hat{\mathbf{v}}_i
   \]
   The initial velocity \( \mathbf{v}_i \) can be broken into components:
   \[
   v_{ix} = v_i \cos(30^\circ)
   \]
   \[
   v_{iy} = v_i \sin(30^\circ)
   \]
   Using \( \cos(30^\circ) = \frac{\sqrt{3}}{2} \) and \( \sin(30^\circ) = \frac{1}{2} \):
   \[
   v_{ix} = 70 \times \frac{\sqrt{3}}{2} = 70 \times 0.866 = 60.62 \, \text{m/s}


Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Special Credits to [GAD-Cell](https://github.com/GAD-cell) for helping Unsloth create this notebook and bringing VLM GRPO into Unsloth!

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>